In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import random

In [7]:
data_file = '../data/heart.csv'
origin_data = pd.read_csv(data_file)

In [9]:
def load_data(data = origin_data, stragy = 2):
    positive_example = data[data['chd'] == 1]
    negitive_example = data[data['chd'] == 0]

    if stragy == 1:
        positive_example = pd.concat([positive_example, positive_example])
    elif stragy == 2: 
        negitive_example = data[data['chd'] == 0]
        negitive_index = random.sample(list(negitive_example.index.values), len(positive_example))
        negitive_example = negitive_example.ix[negitive_index]

    positive_msk = np.random.rand(len(positive_example)) < 0.9
    negitive_msk = np.random.rand(len(negitive_example)) < 0.9
    
    while np.abs(len(positive_example[positive_msk]) - len(negitive_example[negitive_msk])) > 10:
        positive_msk = np.random.rand(len(positive_example)) < 0.9
        negitive_msk = np.random.rand(len(negitive_example)) < 0.9
    
    train_dataset = pd.concat([positive_example[positive_msk], negitive_example[negitive_msk]])
    test_dataset = pd.concat([positive_example[~positive_msk], negitive_example[~negitive_msk]])
    return train_dataset, test_dataset

def normaliztion(dataset):
    return dataset.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=0)

def transfer_famhist(dataset):
    tmp = dataset.replace({'famhist':{'Present':1, 'Absent':0}})
    return normaliztion(tmp).values

def to_one_hotting(data, num_lables=2):
    return (np.arange(num_lables) == data[:,None]).astype(np.float32)

def generate_data(dataset):
    data = dataset.iloc[:,0:9]
    labels = dataset['chd']
    return randomize(transfer_famhist(data), to_one_hotting(labels.values))

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

train_dataset, test_dataset = load_data(origin_data)

train_data, train_label = generate_data(train_dataset)
test_data, test_label = generate_data(test_dataset)

print(train_data.shape, train_label.shape)  

(293, 9) (293, 2)


In [10]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 16
n_epochs = 100

In [12]:
X = tf.placeholder(dtype = np.float32, shape = [None, 9], name='X')
Y = tf.placeholder(dtype = np.float32, shape = [None, 2], name='Y')

W = tf.Variable(tf.random_normal([9, 2]), name='W')
b = tf.Variable(tf.zeros([2]), name='b')

logits = tf.matmul(X, W) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = logits)

loss = tf.reduce_mean(entropy)

preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(len(train_data)/batch_size)
    for i in range(n_epochs): 
        total_loss = 0

        for index in range(n_batches):
            
            X_batch = train_data[index*batch_size:(index+1)*batch_size]
            Y_batch = train_label[index*batch_size:(index+1)*batch_size]
            
            _, loss_batch= sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += loss_batch
            
        test_accuracy = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
        print('Average loss:{0} at epoch:{1}'.format(total_loss/n_batches, i), ' and accuracy is :', test_accuracy / len(test_label))
        
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')
    
    total_accuracy = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
    print('Final Accuracy is :', test_accuracy / len(test_label))

Average loss:0.8450903561380174 at epoch:0  and accuracy is : 0.444444444444
Average loss:0.759326352013482 at epoch:1  and accuracy is : 0.555555555556
Average loss:0.6988345517052544 at epoch:2  and accuracy is : 0.62962962963
Average loss:0.656263111366166 at epoch:3  and accuracy is : 0.592592592593
Average loss:0.6258349120616913 at epoch:4  and accuracy is : 0.62962962963
Average loss:0.6035914503865771 at epoch:5  and accuracy is : 0.592592592593
Average loss:0.587040364742279 at epoch:6  and accuracy is : 0.62962962963
Average loss:0.5745855818192164 at epoch:7  and accuracy is : 0.592592592593
Average loss:0.5651479131645627 at epoch:8  and accuracy is : 0.592592592593
Average loss:0.5579578760597441 at epoch:9  and accuracy is : 0.592592592593
Average loss:0.552448151840104 at epoch:10  and accuracy is : 0.592592592593
Average loss:0.5481946253114276 at epoch:11  and accuracy is : 0.592592592593
Average loss:0.5448801037338045 at epoch:12  and accuracy is : 0.592592592593
Ave